In [9]:
import os
import json
import pandas as pd
import traceback

In [10]:
from langchain.chat_models import ChatOpenAI

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
KEY= os.getenv("OPENAI_API_KEY")

In [13]:
print(KEY)

sk-proj-AyRRf06DfYb5BhwFk5bTT3BlbkFJHEf1qNAjIXMbmvq8fUfY


In [14]:
llm= ChatOpenAI(openai_api_key = KEY, model_name= "gpt-3.5-turbo",temperature=0.5)

c:\Users\makin\mcqsgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
